# Getting started for the AML 2023/2024 Egocentric Vision Project

<a href="https://colab.research.google.com/github/Zohreh-Aghajanloo/Egocentric-AR/blob/master/colab_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading the code

In [ ]:
# Clone the public repository (or your repository)
!git clone https://github.com/Zohreh-Aghajanloo/Egocentric-AR.git aml23-ego

In [ ]:
# Installing missing dependencies
!pip install omegaconf coloredlogs wandb

## EPIC-Kitchens-55 dataset

**READ carefully!**

To develop the project, you need to download the RGB frames for a subset of EPIC-Kitchens-55 (participants P08, P01 and P22) from [here](https://drive.google.com/drive/u/1/folders/1dJOtZ07WovP3YSCRAnU0E4gsfqDzpMVo). 

You also need to the pretrained checkpoints for each domain from [here](https://politoit-my.sharepoint.com/:f:/g/personal/simone_peirone_polito_it/ErdsZhvmR65Lun5_5O0-l5sBTPjCCZZq2f700Tj_CNzjTQ?e=L1yflf).

Add the Google Drive directory containing the dataset to your Google Drive or upload the dataset on your Google Drive to access it from Google Colab.

**NOTE**: As the dataset is quite heavy, we stronly suggest you to implement and test all your code on one for the three dataset. Then, once you are sure everything works, repeat the experiments on the remaining two datasets.

In [ ]:
# Mount google drive 
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%bash
mkdir -p ek_data/frames
mkdir -p aml23-ego/saved_models

cp /content/drive/MyDrive/Egocentric_Action_Recognition/Epic_Kitchens_reduced/*.zip ./ek_data
cp /content/drive/MyDrive/Egocentric_Action_Recognition/I3D_SourceOnlyD1.zip ./aml23-ego/saved_models

unzip ./aml23-ego/saved_models/I3D_SourceOnlyD1.zip -d aml23-ego/saved_models

for file in ./ek_data/*.zip; do
  fn=$(basename $file)
  fn=${fn/.zip/}
  ls -lah $file
  unzip $file -d ek_data/frames
done

## Features extraction

In [ ]:
%%bash

# Replace the following path with the path of your codebase
cd aml23-ego

python save_feat.py name=save_test_dense_k5 \
  split=test \
  save.dense_sampling.RGB=True \
  save.num_frames_per_clip.RGB=5 \
  config=configs/I3D_save_feat.yaml \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames

# If everything is working, you should expect an error message telling you to implement the '_get_val_indices' method in the dataset class.
# Once you have implemented it, you should run the script for the train and test split of the dataset to extract the features.

## Clustering

In [ ]:
%%bash

cd aml23-ego

python clustering.py

## Train Classifier

In [ ]:
%%bash

cd aml23-ego

python train_classifier.py name=classify_uniform_k25_cnn \
  config=configs/default.yaml \
  action=train \
  train.num_frames_per_clip.RGB=25 \
  test.num_frames_per_clip.RGB=25 \
  models.RGB.model=CNN \
  train.dense_sampling.RGB=False \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  dataset.RGB.features_name=save_uniform_k25

## Test

In [ ]:
%%bash

cd aml23-ego

python train_classifier.py name=test_dense_k5_lstm \
  config=configs/default.yaml \
  action=test \
  train.num_frames_per_clip.RGB=5 \
  test.num_frames_per_clip.RGB=5 \
  models.RGB.model=LSTM \
  train.dense_sampling.RGB=True \
  dataset.shift=D1-D1 \
  dataset.RGB.data_path=../ek_data/frames \
  dataset.RGB.features_name=save_dense_k5 \
  resume_from=saved_models/classify_dense_k5_lstm/Jun15_09-14-32/classify_dense_k5_lstm

## ActionNet - Feature Extraction

In [ ]:
%%bash
mkdir -p an_data
mkdir -p aml23-ego/saved_models

cp /content/drive/MyDrive/Egocentric_Action_Recognition/emg_data/* ./aml23-ego/action-net/data
cp /content/drive/MyDrive/Egocentric_Action_Recognition/frames.zip ./an_data
cp /content/drive/MyDrive/Egocentric_Action_Recognition/I3D_SourceOnlyD1.zip ./aml23-ego/saved_models

unzip ./aml23-ego/saved_models/I3D_SourceOnlyD1.zip -d aml23-ego/saved_models

for file in ./an_data/*.zip; do
  fn=$(basename $file)
  fn=${fn/.zip/}
  ls -lah $file
  unzip $file -d an_data
done

In [ ]:
%%bash

cd aml23-ego

python save_feat_actionNet.py name=save_actionNet \
  config=configs/I3D_save_feat.yaml \
  dataset.RGB.data_path=../an_data/frames \
  split=train \
  dataset.shift=D1-D1 \
  dataset.annotations_path=action-net/data \
  dataset.RGB.tmpl="frame_{:010d}.jpg"

## ActionNet - Classification

In [ ]:
%%bash

cd aml23-ego

python train_classifier_actionNet.py name=classify_actionNet_EMG_lstm \
  config=configs/actionNet_classifier.yaml \
  action=train \
  models.EMG.model=EMG_LSTM